This notebook is used to create a World builder file for nuvel plate boundaries (Argus and Gordon, 1991). We use an input file, nuvel_plates.txt which contains line segments describig the plate boundaries.

In [3]:
import json
import os
import numpy as np

In [2]:
# This cell prints the number of line segments in the input nuvel_faults.txt 
# downloaded from https://www.earthbyte.org/plate-boundaries/ .
# In the input file, ">" separates the line segments and we use that separator
# in awk to find out number of line segments which are the number of faults for 
# the worldbuilder.

file_name        = "nuvel_plates.txt"
number_of_faults = os.popen('''awk 'BEGIN {RS=">"} END{print NR}' ''' + file_name).read()
number_of_faults = int(number_of_faults)

In [4]:
# This function defines the characteristics of the input faults such as the 
# name, coordinates (used from the input file), dip (vertical), thickness
# and length are defined appropriately for our global models.
def make_fault_json(fault_name, coordinates):
    fault_json = {}
    fault_json['geometry'] = {}
    
    out = {
    "model": "fault",
    "name": fault_name,
    "coordinates": coordinates,
    "dip point": [0, 1], #get_dip_point(feat),
    "min depth": -1e3,
    "max depth": 300e3,
    "segments": [
        {
            "length": 300e3,
            "thickness": [100e3],
            "angle": [90, 90] 
        }
    ],
        
    # temperature, composition, etc. would go here"
    # max depth chosen as the max continental plate thickness
    # hard coded the composition here
    "composition models": [{"model":"smooth", "compositions":4, "side distance fault center":50e3}] 
     }
    
    return out

In [16]:
#--------------------------------------------------------------------
# This cell formats data so that the input coordinates are strings 
# in a list with each list item correspoding to 'long lat'.
# Awk initializes with 1 unlike python which has zero-index numbering. 
# The first record does not exist, therefore start with the 2 until 
# the number of faults + 1 for calling line segments in awk.
# After formatting the coordinates from the input file, we store each
# fault feature according to the make_fault_json() function to get
# faults as needed in the world builder format
#--------------------------------------------------------------------

faults_coordinates = []
all_faults_json = []

for i in range (2, number_of_faults + 1):

    faults_coordinates = os.popen('''awk 'BEGIN {FS=" "; OFS=","; RS=">"} {$1=$1}1 {if (NR==%s) print $0}' ''' \
                         % (str(i)) + file_name).read()
    # This is modification of the input strings in the coordinates to floats.
    coordinates_list   = (faults_coordinates.rstrip('\n')).split(',')  
    list_of_floats     = [float(item) for item in coordinates_list]
    # Since we want to modify the longitudes to -180 to 180 for world builder,
    # we separate the latitudes and longitudes. The first value is the longitude
    # and use the value after every 2 values to get all the longitudes from the 
    # longitude, latitude pair.
    longitudes         = list_of_floats[0::2]
    latitudes          = list_of_floats[1::2]
    longitudes_array   = np.array(longitudes)
    
     # We do this to join the faults on the other side of the great circle
    if (i == 31 or i == 21 or i == 20):
        longitudes_array [longitudes_array < 180] = longitudes_array [longitudes_array < 180] - 360
        
    # Convert longitudes to -180 and 180 from 0 to 360.
    longitudes_array [longitudes_array >= 180] = longitudes_array[longitudes_array >= 180] - 360
    
    # These are the desired formatted coordinates for each fault segment
    coordinates_array  = [[longitudes_array[n],latitudes[n]] for n in range(len(longitudes))] 
    
    all_faults_json.append(make_fault_json('fault_' + str(i), coordinates_array))

In [17]:
# This code cell adds the header of the worldbuilder file, and the next cell writes 
# everything into the worldbuilder file.
all_faults_json = {
    "version": "0.5",
    "interpolation": "continuous monotone spline",
    "coordinate system":{"model":"spherical", "depth method":"starting point"},
    "features": all_faults_json
}

In [18]:
with open("./world_builder_nuvel_boundaries.json", "w") as f:
    json.dump(all_faults_json, f)

In [5]:
#-----------------------------------------------------------------------------------
# The next 3 code cells are relevant for using different compositions at the ridges 
# and trenches. We manually use the index for each ridge and trench line segment 
# in the input nuvel_faults.txt file and use those indices to modify the composition
# in the generated world builder file. The modified composition allows us to assign
# different viscosity at the ridges and trenches. Transform faults are also part of
# ridges in our case.
#-----------------------------------------------------------------------------------
ridge_numbers  = [2, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 17, 21, 22, 24, 25, 20, 32]
trench_numbers = [3, 6, 7, 16, 18, 19, 20, 23, 26, 27, 28, 29, 31, 33, 34, 35, 36, 37]

In [8]:
file_name = "./world_builder_nuvel_boundaries.json"
with open(file_name) as f:
    gg = json.load(f)

In [41]:
for i in range (2, number_of_faults + 1):
    fault_name = 'fault_' + str(i)
    if (i in ridge_numbers):
        gg['features'][i-2]['composition models'][0]['compositions'] = 4
    if (i in trench_numbers):
        gg['features'][i-2]['composition models'][0]['compositions'] = 5

In [43]:
with open("./world_builder_nuvel_ridges_trenches_boundaries.json", "w") as f:
    json.dump(gg, f)